In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import torch

In [ ]:
# !wget http://efrosgans.eecs.berkeley.edu/pix2pix/datasets/facades.tar.gz
# !tar -xvf facades.tar.gz

In [ ]:
!pwd

/content


In [ ]:
import torch
from torchvision import transforms
import os, sys
import argparse

In [ ]:
SCRIPT_DIR = os.path.dirname(os.path.abspath("/content"))
sys.path.append(os.path.dirname(SCRIPT_DIR))
SCRIPT_DIR = os.path.dirname(os.path.abspath("/content/code"))
sys.path.append(os.path.dirname(SCRIPT_DIR))
sys.path.append(".")

from datasets import DatasetFromFolder
from model import Generator, Discriminator
import utils

In [ ]:
params = argparse.Namespace()
params.dataset = "facades"
params.direction = 'BtoA'
params.batch_size = 1
params.ngf = 64
params.ndf = 64
params.input_size = 256
params.resize_scale =286
params.crop_size = 256
params.fliplr = True
params.num_epochs = 20
params.lrG = 0.0002
params.lrD = 0.0002
params.lamb = 100.0
params.beta1 = 0.5
params.beta2 = 0.999
params.data_dir = "/content/facades"

In [ ]:
transform = transforms.Compose([transforms.Resize(params.input_size),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))]
                               )
# Train data
train_data = DatasetFromFolder(params.data_dir, subfolder='train', direction=params.direction, transform=transform,
                               resize_scale=params.resize_scale, crop_size=params.crop_size, fliplr=params.fliplr)
train_data_loader = torch.utils.data.DataLoader(dataset=train_data,
                                                batch_size=params.batch_size,
                                                shuffle=True)

# Test data
test_data = DatasetFromFolder(params.data_dir, subfolder='test', direction=params.direction, transform=transform)
test_data_loader = torch.utils.data.DataLoader(dataset=test_data,
                                               batch_size=params.batch_size,
                                               shuffle=False)
test_input, test_target = test_data_loader.__iter__().__next__()

In [ ]:
G = Generator(3, params.ngf, 3)
D = Discriminator(6, params.ndf, 1)
G.cuda()
D.cuda()
G.normal_weight_init(mean=0.0, std=0.02)
D.normal_weight_init(mean=0.0, std=0.02)


/content/model.py:102: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  torch.nn.init.normal(m.conv.weight, mean, std)
/content/model.py:104: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  torch.nn.init.normal(m.deconv.weight, mean, std)
/content/model.py:131: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  torch.nn.init.normal(m.conv.weight, mean, std)


In [ ]:
# Loss function
BCE_loss = torch.nn.BCELoss().cuda()
L1_loss = torch.nn.L1Loss().cuda()

# Optimizers
G_optimizer = torch.optim.Adam(G.parameters(), lr=params.lrG, betas=(params.beta1, params.beta2))
D_optimizer = torch.optim.Adam(D.parameters(), lr=params.lrD, betas=(params.beta1, params.beta2))




In [ ]:
# Training GAN
D_avg_losses = []
G_avg_losses = []

step = 0
for epoch in range(params.num_epochs):
    D_losses = []
    G_losses = []

    # training
    for i, (input, target) in enumerate(train_data_loader):

        # input & target image data
        x_ = input.cuda()
        y_ = target.cuda()

        # Train discriminator with real data
        D_real_decision = D(x_, y_).squeeze()
        real_ = torch.ones(D_real_decision.size()).cuda()
        D_real_loss = BCE_loss(D_real_decision, real_)

        # Train discriminator with fake data
        gen_image = G(x_)
        D_fake_decision = D(x_, gen_image).squeeze()
        fake_ = torch.zeros(D_fake_decision.size()).cuda()
        D_fake_loss = BCE_loss(D_fake_decision, fake_)

        # Back propagation
        D_loss = (D_real_loss + D_fake_loss) * 0.5
        D.zero_grad()
        D_loss.backward()
        D_optimizer.step()

        # Train generator
        gen_image = G(x_)
        D_fake_decision = D(x_, gen_image).squeeze()
        G_fake_loss = BCE_loss(D_fake_decision, real_)

        # L1 loss
        l1_loss = params.lamb * L1_loss(gen_image, y_)

        # Back propagation
        G_loss = G_fake_loss + l1_loss
        G.zero_grad()
        G_loss.backward()
        G_optimizer.step()

        # loss values
        D_losses.append(D_loss.item())
        G_losses.append(G_loss.item())

        print('Epoch [%d/%d], Step [%d/%d], D_loss: %.4f, G_loss: %.4f'
              % (epoch+1, params.num_epochs, i+1, len(train_data_loader), D_loss.item(), G_loss.item()))
        
        step += 1

    D_avg_loss = torch.mean(torch.FloatTensor(D_losses))
    G_avg_loss = torch.mean(torch.FloatTensor(G_losses))

    # avg loss values for plot
    D_avg_losses.append(D_avg_loss)
    G_avg_losses.append(G_avg_loss)

    # Show result for test image
    gen_image = G(test_input.cuda())
    gen_image = gen_image.cpu().data
    utils.plot_test_result(test_input, test_target, gen_image, epoch, save=True)

# Plot average losses
utils.plot_loss(D_avg_losses, G_avg_losses, params.num_epochs, save=True)


In [ ]:
utils.make_gif(params.dataset, params.num_epochs, save_dir="./results/")

In [ ]:
!zip -r all.zip *

  adding: code/ (stored 0%)
  adding: code/__init__.py (stored 0%)
  adding: code/.ipynb_checkpoints/ (stored 0%)
  adding: code/__pycache__/ (stored 0%)
  adding: code/__pycache__/__init__.cpython-38.pyc (deflated 30%)
  adding: datasets.py (deflated 70%)
  adding: __init__.py (stored 0%)
  adding: model.py (deflated 80%)
  adding: __pycache__/ (stored 0%)
  adding: __pycache__/utils.cpython-38.pyc (deflated 39%)
  adding: __pycache__/__init__.cpython-38.pyc (deflated 30%)
  adding: __pycache__/model.cpython-38.pyc (deflated 57%)
  adding: __pycache__/datasets.cpython-38.pyc (deflated 44%)
  adding: results/ (stored 0%)
  adding: results/Result_epoch_5.png (deflated 2%)
  adding: results/Result_epoch_3.png (deflated 3%)
  adding: results/Result_epoch_7.png (deflated 2%)
  adding: results/Result_epoch_12.png (deflated 2%)
  adding: results/Result_epoch_6.png (deflated 2%)
  adding: results/Result_epoch_11.png (deflated 2%)
  adding: results/facades_pix2pix_epochs_20.gif (deflated 8%)
 